In [1]:
#Praca Licencjacka 2025

In [ ]:
# Instalacja wymaganych bibliotek
%pip install -r requirements.txt

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, spearmanr, jarque_bera
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
#Dane
data_path = "Model_Lekarz.xlsx" 
data = pd.read_excel(data_path)
data.head()

In [5]:
#Przypisywanie kategorii i etykiet
data['miejscowosc'] = pd.Categorical(data['klasa_miejsca'], 
                                     categories=[1, 2, 3, 4, 5, 6], 
                                     ordered=True)
data['miejscowosc'] = data['miejscowosc'].cat.rename_categories(["powyzej 500 tys", "200-499 tys", 
                                                                 "100-199 tys", "20-99 tys", 
                                                                 "ponizej 20 tys", "wies"])

data['wyksztalcenie_1'] = pd.Categorical(data['wyksz'], 
                                       categories=[1, 2, 3], 
                                       ordered=True)
data['wyksztalcenie_1'] = data['wyksztalcenie_1'].cat.rename_categories(["podstawowe", "srednie", 
                                                                     "wyzsze"])

data['mozliwosci_oszczedzania_1'] = pd.Categorical(data['moz_oszcz'], 
                                                 categories=[1, 2, 3, 4], 
                                                 ordered=True)
data['mozliwosci_oszczedzania_1'] = data['mozliwosci_oszczedzania_1'].cat.rename_categories(["regularnie", "nieregularnie", 
                                                                                         "mozemy_ale_nie", "nie_mozemy"])

data['zrodlo_utrzymania_1'] = pd.Categorical(data['glowne_zrod_utrzym'], 
                                           categories=[1, 2, 3, 4, 5], 
                                           ordered=True)
data['zrodlo_utrzymania_1'] = data['zrodlo_utrzymania_1'].cat.rename_categories(["pracownicy", "rolnicy", 
                                                                             "praca na wlasny rachunek", "emeryt/rencista", 
                                                                             "niezarobkowe zrodla dochodu"])

data['sytuacja_materialna_1'] = pd.Categorical(data['syt_mat'], 
                                             categories=[1, 2, 3, 4, 5], 
                                             ordered=True)
data['sytuacja_materialna_1'] = data['sytuacja_materialna_1'].cat.rename_categories(["dobra", "raczej dobra", 
                                                                                 "przecietna", "raczej zla", "zla"])





# Filtrowanie danych
model2 = data[data['wyd_os_lek'] > 0]

In [ ]:
model2.head()


In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Histogram wydatków
ax1.hist(model2['wyd_os_lek'], bins=30, color='blue', edgecolor='purple')
ax1.set_xlabel('Wydatki')
ax1.set_title('Histogram wydatków')

# Histogram logarytmu wydatków
ax2.hist(np.log(model2['wyd_os_lek']), bins=30, color='blue', edgecolor='purple')
ax2.set_xlabel('Log wydatków')
ax2.set_title('Histogram logarytmu wydatków')

plt.tight_layout()
plt.show()

In [ ]:
summary_wydatki_los = model2['wyd_os_lek'].describe()
print(summary_wydatki_los)

In [ ]:
#Miejscowość
sns.boxplot(y=model2['miejscowosc'], x=np.log(model2['wyd_os_lek']), orient='h')
plt.title('Wydatki vs miejscowość')
plt.xlabel('Logarytm wydatków')
plt.ylabel('Miejscowość')
plt.show()

In [ ]:
summary_miejscowosc = model2.groupby('miejscowosc', observed = 'False')['wyd_os_lek'].describe()
print(summary_miejscowosc)

In [ ]:
#Wykształcenie
sns.boxplot(y=model2['wyksztalcenie_1'], x=np.log(model2['wyd_os_lek']), orient='h')
plt.title('Wydatki vs wykształcenie')
plt.xlabel('Logarytm wydatków')
plt.ylabel('Wykształcenie')
plt.show()

In [ ]:
summary_wyksztalcenie = model2.groupby('wyksztalcenie_1', observed = 'False')['wyd_os_lek'].describe()
print(summary_wyksztalcenie)


In [ ]:
from scipy.stats import kruskal

# Dane do testu – oddzielne grupy
podstawowe = model2[model2['wyksztalcenie_1'] == 'podstawowe']['wyd_os_lek']
srednie = model2[model2['wyksztalcenie_1'] == 'srednie']['wyd_os_lek']
wyzsze = model2[model2['wyksztalcenie_1'] == 'wyzsze']['wyd_os_lek']

# Test Kruskala-Wallisa
stat, p = kruskal(podstawowe, srednie, wyzsze)

print(f"Wartość statystyki H = {stat:.4f}")
print(f"Wartość p = {p:.4f}")

if p < 0.05:
    print("Różnice pomiędzy grupami są statystycznie istotne (p < 0.05).")
else:
    print("Brak istotnych statystycznie różnic pomiędzy grupami.")

In [ ]:
#Histogram dochodu
model2['dochod_los'] = model2['doch_os'].apply(lambda x: 0.1 if (x == 0 or x<0) else x)
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].hist(model2['dochod_los'], bins=30, color='blue', edgecolor = 'purple')
ax[0].set_title('Histogram dochodu na osobę')
ax[0].set_xlabel('Dochód na osobę')

ax[1].hist(np.log(model2['dochod_los']), bins=30, color='blue', edgecolor = 'purple')
ax[1].set_title('Histogram logarytmu dochodu na osobę')
ax[1].set_xlabel('Logarytm dochodu na osobę')

plt.tight_layout()
plt.show()

In [ ]:
model2['log_dochod'] = np.log(model2['dochod_los'])
model2['log_dochod_kwadrat'] = model2['log_dochod'] ** 2

import statsmodels.formula.api as smf

model = smf.ols('wyd_os_lek ~ log_dochod + log_dochod_kwadrat', data=model2).fit()
print(model.summary())

In [ ]:
summary_dochod_los = model2['doch_os'].describe()
print(summary_dochod_los)

In [ ]:
#Wykres rozrzutu
plt.scatter(model2['wiek'], model2['wyd_os_lek'], alpha=0.5)
plt.title('Wydatki vs Wiek')
plt.xlabel('Wiek')
plt.ylabel('Wydatki')
plt.show()

# Dla logarytmów
plt.scatter(model2['wiek'], np.log(model2['wyd_os_lek']), alpha=0.5)
plt.title('Logarytm wydatków vs Wiek')
plt.xlabel('Wiek')
plt.ylabel('Logarytm wydatków')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Wczytanie danych z pliku .xlsx
model2 = pd.read_excel('Model_Lekarz.xlsx')  

# Tworzenie zmiennych zero-jedynkowych dla wykształcenia
model2['wyzsze'] = model2['wyksz'].apply(lambda x: 1 if x == 3 else 0)
model2['srednie'] = model2['wyksz'].apply(lambda x: 1 if x == 2  else 0)
model2['podstawowe'] = model2['wyksz'].apply(lambda x: 1 if x == 1 else 0)
print(model2[['wyzsze', 'srednie', 'podstawowe']].sum())

# Miasto
model2['powyzej500tys'] = model2['klasa_miejsca'].apply(lambda x: 1 if x == 1 else 0)
model2['od200do499'] = model2['klasa_miejsca'].apply(lambda x: 1 if x == 2 else 0)
model2['od100do199'] = model2['klasa_miejsca'].apply(lambda x: 1 if x == 3 else 0)
model2['od20do99'] = model2['klasa_miejsca'].apply(lambda x: 1 if x == 4 else 0)
model2['ponizej20'] = model2['klasa_miejsca'].apply(lambda x: 1 if x == 5 else 0)
model2['wies'] = model2['klasa_miejsca'].apply(lambda x: 1 if x == 6 else 0)
print(model2[['powyzej500tys', 'od200do499', 'od100do199', 'od20do99', 'ponizej20', 'wies']].sum())

# Możliwości oszczędzania
model2['regularnie'] = model2['moz_oszcz'].apply(lambda x: 1 if x == 1 else 0)
model2['nieregularnie'] = model2['moz_oszcz'].apply(lambda x: 1 if x == 2 else 0)
model2['mozemy_ale_nie'] = model2['moz_oszcz'].apply(lambda x: 1 if x == 3 else 0)
model2['nie_mozemy'] = model2['moz_oszcz'].apply(lambda x: 1 if x == 4 else 0)
print(model2[['regularnie', 'nieregularnie', 'mozemy_ale_nie', 'nie_mozemy']].sum())

# Sytuacja materialna
model2['dobra'] = model2['syt_mat'].apply(lambda x: 1 if x == 1 else 0)
model2['raczej_dobra'] = model2['syt_mat'].apply(lambda x: 1 if x == 2 else 0)
model2['przecietna'] = model2['syt_mat'].apply(lambda x: 1 if x == 3 else 0)
model2['raczej_zla'] = model2['syt_mat'].apply(lambda x: 1 if x == 4 else 0)
model2['zla'] = model2['syt_mat'].apply(lambda x: 1 if x == 5 else 0)
print(model2[['dobra', 'raczej_dobra', 'przecietna', 'raczej_zla', 'zla']].sum())

# Źródło utrzymania
model2['pracownicy'] = model2['glowne_zrod_utrzym'].apply(lambda x: 1 if x == 1 else 0)
model2['rolnicy'] = model2['glowne_zrod_utrzym'].apply(lambda x: 1 if x == 2 else 0)
model2['wlasny_rachunek'] = model2['glowne_zrod_utrzym'].apply(lambda x: 1 if x == 3 else 0)
model2['emeryt_rencista'] = model2['glowne_zrod_utrzym'].apply(lambda x: 1 if x == 4 else 0)
model2['niezarobkowe'] = model2['glowne_zrod_utrzym'].apply(lambda x: 1 if x == 5 else 0)
print(model2[['pracownicy', 'rolnicy', 'wlasny_rachunek', 'emeryt_rencista', 'niezarobkowe']].sum())


In [ ]:
#Macierz korelacji
first_12_columns = model2.iloc[:, :14]

# Obliczamy macierz korelacji
correlation_matrix = first_12_columns.corr()

# Wyświetlamy macierz korelacji jako wykres
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Macierz korelacji')
plt.show()

In [ ]:
import statsmodels.formula.api as smf


# Zmienna zależna (logarytm wydatków)
model2['log_wydatki'] = model2['wyd_os_lek'].apply(lambda x: np.log(x) if x > 0 else np.nan)

# Formuła regresji
formula = "log_wydatki ~ wiek + doch_os + wyzsze + srednie + powyzej500tys + od200do499 + od100do199 + od20do99 + ponizej20 + rolnicy + wlasny_rachunek + emeryt_rencista + niezarobkowe + liczba_osob + regularnie + nieregularnie + mozemy_ale_nie +  plec + inwalidztwo"

# Regresja
reg_model2 = smf.ols(formula=formula, data=model2).fit()

# Wyniki
print(reg_model2.summary())



In [21]:

#Model aby zaobserwować obserwacje odstające

In [22]:
#odstajace

In [ ]:
import stargazer
print(stargazer.__file__)
print(dir(stargazer))

In [ ]:
#levare point 
h_lev = (2*len(reg_model2.params))/reg_model2.nobs
round(h_lev, 2)

In [ ]:
sm.graphics.influence_plot(reg_model2, criterion = 'Cooks')
plt.axvline(x = round(h_lev, 2), color = 'g')
plt.axhline(y = 2, color = 'r')
plt.axhline(y = -2, color = 'r')
plt.show()

In [26]:
model3 = model2.drop(index=[771, 975, 1169, 881, 740, 974, 842, 482, 943, 603])

In [ ]:
import statsmodels.formula.api as smf


# Zmienna zależna (logarytm wydatków)
model3['log_wydatki'] = model3['wyd_os_lek'].apply(lambda x: np.log(x) if x > 0 else np.nan)

model3['dochod_los'] = model3['doch_os'].apply(lambda x: 0.1 if (x == 0 or x<0) else x)

# Formuła regresji
formula = "np.log(log_wydatki) ~ np.log(dochod_los) + wyzsze + srednie + powyzej500tys + od200do499 + od100do199 + od20do99 + ponizej20 + liczba_osob +I(liczba_osob**2) + I(liczba_osob**3)  + inwalidztwo"

# Regresja
reg_2 = smf.ols(formula=formula, data=model3).fit()

# Wyniki
print(reg_2.summary())





In [ ]:
#Test Reset 
#najważniejsze założenie - czy jest to regresja liniowa 
model3['residuals'] = reg_2.resid

model3['predictions'] = reg_2.predict()

sns.residplot(data = model3, x = 'predictions' , y = 'residuals', line_kws = dict(color = 'g'))

sns.residplot(data = model3, x = 'predictions' , y = 'residuals', lowess = True, line_kws = dict(color = 'g'))
#ta zielona linia jest rodzajem spłaczczenia wykresu rozrzutu - nam zależy na tym aby ta zielona linia polrywała się z linią kropkowaną.
#Idealnia jak ta zielona się pokrywa z kropkowaną 

In [29]:
#Zielona linia nie jest zupełnie płaska – delikatnie się wygina, co może sugerować nieliniowość, poprawa do 44% reset przy pomocy logarytmu i zmiennych do kwadratu

# Można zauważyć wachlarzowaty kształt – wariancja reszt nie jest całkiem stała (homoskedastyczna). Przy większych wartościach predykcji rozrzut się zwiększa → możliwa heteroskedastyczność.#
#Następnie testy na heteroskedastycznosc - najprawdopodobniej uzycie macierzy odpornej 

In [ ]:
#sam test reset
import statsmodels.stats.api as sms
sms.linear_reset(reg_2, power = 4, test_type ='fitted')

In [ ]:
#Heteroskedastyczność
plt.scatter(reg_2.fittedvalues, reg_2.resid)
plt.axhline(y = 0, color = 'red')
plt.xlabel('fitted values')
plt.ylabel('residuals')
#więcej zaznaczonych obszarów na dole wykresu rozrzutu - podejrzenie o homoskedastycznosci

In [ ]:
plt.scatter(reg_2.fittedvalues, reg_2.resid)
plt.axhline(y = 0, color = 'red')
plt.axhline(y = 0 + 3*reg_2.resid.std(), color = 'green')
plt.axhline(y = 0 - 3*reg_2.resid.std(), color = 'green')
plt.xlabel('fitted values')
plt.ylabel('residuals')

In [ ]:
#test B-P
test1 = sms.het_breuschpagan(reg_2.resid, reg_2.model.exog)
test1
#Odrzucam hipotezę zerowoą o homoskedastycznosci

In [ ]:
test2 = sms.het_white(reg_2.resid, reg_2.model.exog)
test2
#nie zdaje Whitey'a pomimo prob poprawy jak dodanie poteg, czy logarytmu, zatem stosuję macierz odporną 

In [35]:
#Heteroskedastycznosc, zatem uzywam macierzy odpornej 

In [36]:
#Macierz Odporna 

In [ ]:
import statsmodels.formula.api as smf


# Zmienna zależna (logarytm wydatków)
model3['log_wydatki'] = model3['wyd_os_lek'].apply(lambda x: np.log(x) if x > 0 else np.nan)

model3['dochod_los'] = model3['doch_os'].apply(lambda x: 0.1 if (x == 0 or x<0) else x)

# Formuła regresji
formula = "np.log(log_wydatki) ~ np.log(dochod_los) + wyzsze + srednie + powyzej500tys + od200do499 + od100do199 + od20do99 + ponizej20 + liczba_osob +I(liczba_osob**2) + I(liczba_osob**3)  + inwalidztwo"
robust_reg_3 = smf.ols(formula=formula, data=model3).fit(cov_type = 'HC0')
print(robust_reg_3.summary())

In [ ]:
#sam test reset
import statsmodels.stats.api as sms
sms.linear_reset(robust_reg_3, power = 4, test_type ='fitted')

In [39]:
#VIF

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import patsy

# Przygotowanie macierzy projektowej na podstawie formuły modelu
y, X = patsy.dmatrices(formula, data=model3, return_type="dataframe")

# Obliczanie współczynników VIF
vif_data = pd.DataFrame({
    "Variable": X.columns,
    "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
})

# Wyświetlenie tabeli
print(vif_data)

In [41]:
#Vify bez przekształceń zmiennych poniżej 10 - w porządku 

In [42]:
#Test J-B
from statsmodels.stats.stattools import jarque_bera

In [43]:
residuals_j_b = robust_reg_3.resid

In [ ]:
jb_test_stat, jb_pvalue, skew, kurtosis = jarque_bera(residuals_j_b)
print(f"Statystyka testu: {jb_test_stat}, p-wartość: {jb_pvalue}")

In [ ]:
import scipy.stats as stats
# Tworzymy wykres QQ
plt.figure(figsize=(6,6))
stats.probplot(residuals_j_b, dist="norm", plot=plt)

# Wyświetlamy wykres
plt.title('Wykres QQ reszt')
plt.show()

In [46]:
#W środkowym zakresie (od ok. -2 do +2) reszty dobrze pokrywają się z linią → rozkład jest bliski normalnemu.

#Na krańcach (zarówno dla niskich, jak i wysokich wartości) widać niewielkie odchylenia od czerwonej linii – to typowe i nie musi od razu świadczyć o dużym problemie.

#Dane empiryczne często mają grubsze lub cieńsze ogony --> Mam ponad 1000 obserwacji dzięki Centralnemu Twierdzeniu Granicznego (CTG) --> zakładam noramalność rozkłądu

In [ ]:
import pandas as pd
from scipy.stats import anderson, normaltest

def perform_normality_tests(series):
    """
    Wykonuje dwa testy normalności: Andersona-Darlinga oraz test D'Agostino-Pearsona.
    
    Args:
        series (pd.Series): Kolumna danych.
        
    Returns:
        dict: Wyniki testów.
    """
    # Usuwamy brakujące dane
    values = series.dropna()
    if len(values) < 3:
        return None

    # Test Andersona-Darlinga
    ad_result = anderson(values)
    
    # Test D'Agostino-Pearsona
    stat_np, p_np = normaltest(values)
    
    return {
        'anderson_statistic': ad_result.statistic,
        'anderson_critical_values': ad_result.critical_values,
        'anderson_significance_levels': ad_result.significance_level,
        'dagostino_statistic': stat_np,
        'dagostino_pvalue': p_np
    }

# Wczytanie danych (upewnij się, że ścieżka do pliku jest poprawna)
data_path = "Model_Lekarz.xlsx"
data = pd.read_excel(data_path)

# Wybieramy kolumnę do testu, np. 'wyd_os_lek'
column_to_test = "wyd_os_lek"
results = perform_normality_tests(data[column_to_test])

# Prezentacja wyników
print("Wyniki testu Andersona-Darlinga:")
print(f"Statystyka: {results['anderson_statistic']:.4f}")
print("Krytyczne wartości i poziomy istotności:")
for crit_val, sig_level in zip(results['anderson_critical_values'], results['anderson_significance_levels']):
    print(f"   Poziom istotności {sig_level}%: {crit_val:.4f}")
    
print("\nWyniki testu D'Agostino-Pearsona:")
print(f"Statystyka: {results['dagostino_statistic']:.4f}")
print(f"P-value: {results['dagostino_pvalue']:.4f}")

# Interpretacja:
if results['dagostino_pvalue'] > 0.05:
    print("\nD'Agostino-Pearsona: Dane są zgodne z rozkładem normalnym.")
else:
    print("\nD'Agostino-Pearsona: Dane nie są zgodne z rozkładem normalnym.")